<a href="https://colab.research.google.com/github/atharvadesai1/BE-Project-Codes/blob/main/alzheimer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.layers import BatchNormalization
import cv2
import zipfile

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!unzip "/content/drive/MyDrive/alzheimer_dataset.zip" -d "/content/dataset"

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13 (47).jpg  
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13 (48).jpg  
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13 (49).jpg  
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13 (5).jpg  
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13 (50).jpg  
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13 (51).jpg  
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13 (52).jpg  
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13 (53).jpg  
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13 (54).jpg  
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13 (55).jpg  
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13 (56).jpg  
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13 (57).jpg  
  inflating: /content/dataset/alzheimer_dataset/NonDemented/13

In [3]:
# Step 1: Define Constants
CLASSES = ['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented']
NUM_CLASSES = len(CLASSES)
IMG_SIZE = 176
IMAGE_SIZE = [IMG_SIZE, IMG_SIZE]
BATCH_SIZE = 32
EPOCHS = 50
MODEL_PATH = '/content/drive/MyDrive/alzheimer_resnet50.h5'

In [4]:
# Now set the base directory to the extracted folder
base_dir = '/content/dataset/alzheimer_dataset'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

In [5]:
# Create train/validation split if not already exists
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    os.makedirs(val_dir)

    for class_name in CLASSES:
        os.makedirs(os.path.join(train_dir, class_name))
        os.makedirs(os.path.join(val_dir, class_name))

        # Get images for current class
        src_dir = os.path.join(base_dir, class_name)
        images = [f for f in os.listdir(src_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
        np.random.shuffle(images)
        split_idx = int(0.8 * len(images))

        # Move to train and validation
        for img in images[:split_idx]:
            os.rename(os.path.join(src_dir, img),
                     os.path.join(train_dir, class_name, img))

        for img in images[split_idx:]:
            os.rename(os.path.join(src_dir, img),
                     os.path.join(val_dir, class_name, img))

In [6]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
# Flow images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 5119 images belonging to 4 classes.
Found 1281 images belonging to 4 classes.


In [8]:
# Build ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# Callbacks
callbacks = [
    ModelCheckpoint(MODEL_PATH, monitor='val_accuracy', save_best_only=True, mode='max'),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

In [13]:
# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step - accuracy: 0.3357 - loss: 1.7721

159/159 ━━━━━━━━━━━━━━━━━━━━ 63s 398ms/step - accuracy: 0.3357 - loss: 1.7718 - val_accuracy: 0.5000 - val_loss: 1.1622
Epoch 2/50
  1/159 ━━━━━━━━━━━━━━━━━━━━ 13s 85ms/step - accuracy: 0.4062 - loss: 1.4173

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.4062 - loss: 1.4173 - val_accuracy: 0.5000 - val_loss: 1.1635
Epoch 3/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step - accuracy: 0.3410 - loss: 1.6560

159/159 ━━━━━━━━━━━━━━━━━━━━ 84s 530ms/step - accuracy: 0.3412 - loss: 1.6555 - val_accuracy: 0.5258 - val_loss: 1.1059
Epoch 4/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.4688 - loss: 1.2796 - val_accuracy: 0.5211 - val_loss: 1.1092
Epoch 5/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 142s 546ms/step - accuracy: 0.4089 - loss: 1.4685 - val_accuracy: 0.5250 - val_loss: 1.0494
Epoch 6/50
  1/159 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.4688 - loss: 1.1707

159/159 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.4688 - loss: 1.1707 - val_accuracy: 0.5328 - val_loss: 1.0358
Epoch 7/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 46s 289ms/step - accuracy: 0.4105 - loss: 1.4192 - val_accuracy: 0.4648 - val_loss: 1.1830
Epoch 8/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5000 - loss: 1.2239 - val_accuracy: 0.4680 - val_loss: 1.1752
Epoch 9/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 82s 307ms/step - accuracy: 0.4415 - loss: 1.3752 - val_accuracy: 0.5219 - val_loss: 1.1216
Epoch 10/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.3438 - loss: 1.5415 - val_accuracy: 0.5227 - val_loss: 1.1280
Epoch 11/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 45s 281ms/step - accuracy: 0.4393 - loss: 1.3407 - val_accuracy: 0.5234 - val_loss: 1.0876


In [ ]:
# Step 4: Evaluate the Model
# Load the best saved model
model = load_model(MODEL_PATH)

In [ ]:
# Evaluate on validation set
val_loss, val_acc = model.evaluate(validation_generator)
print(f'\nValidation Accuracy: {val_acc*100:.2f}%')

In [ ]:
# Generate predictions
validation_generator.reset()
y_pred = model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = validation_generator.classes

In [ ]:
# Classification report
print('\nClassification Report:')
print(classification_report(y_true, y_pred_classes, target_names=CLASSES))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=CLASSES, yticklabels=CLASSES)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Step 5: Grad-CAM Implementation
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
def display_gradcam(img_path, heatmap, alpha=0.4):
    # Load the original image
    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = plt.cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    # Display Grad CAM
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img / 255.)
    plt.title("Original")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(superimposed_img)
    plt.title("Grad-CAM")
    plt.axis('off')
    plt.show()

In [ ]:
# Get a sample image for Grad-CAM
sample_img_path = os.path.join(val_dir, CLASSES[1], os.listdir(os.path.join(val_dir, CLASSES[1]))[0])
img = tf.keras.preprocessing.image.load_img(sample_img_path, target_size=IMAGE_SIZE)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.

In [ ]:
# Generate class activation heatmap
last_conv_layer_name = "conv5_block3_out"
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

In [ ]:
# Generate class activation heatmap
last_conv_layer_name = "conv5_block3_out"
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

In [ ]:
# Display heatmap
display_gradcam(sample_img_path, heatmap)

In [ ]:
# Step 6: Test with Custom Input
def predict_custom_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMAGE_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.

    # Make prediction
    pred = model.predict(img_array)
    pred_class = CLASSES[np.argmax(pred)]
    confidence = np.max(pred) * 100

    # Display image and prediction
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Prediction: {pred_class}\nConfidence: {confidence:.2f}%')
    plt.show()

    # Generate and display Grad-CAM
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
    display_gradcam(image_path, heatmap)

    return pred_class, confidence

In [ ]:
# Example usage with a custom image
# Upload an image file first
from google.colab import files
uploaded = files.upload()
custom_image_path = list(uploaded.keys())[0]
predicted_class, confidence = predict_custom_image(custom_image_path)
print(f"\nFinal Prediction: {predicted_class} with {confidence:.2f}% confidence")